In [ ]:
from binary_tree import TreeNode
from typing import Optional, List
from collections import deque
import math
from math import inf
import collections
import numpy as np
from collections import defaultdict
import itertools
import bisect


## 二叉树

In [ ]:


class Solution:
    def maxDepth(self, root: Optional[TreeNode]) -> int:
        if not root.val:
            return 0
        return max(self.maxDepth(root.left), self.maxDepth(root.right)) + 1

    def invertTree(self, root: Optional[TreeNode]) -> Optional[TreeNode]:
        if not root:
            return
        sub_left = self.invertTree(root.left)
        sub_right = self.invertTree(root.right)

        root.left = sub_right
        root.right = sub_left

        return root

    def isSymmetric(self, root):
        def recursive_check(left, right):
            if left is None and right is None:
                return True
            if left is None or right is None:
                return False
            if left.val != right.val:
                return False
            else:
                return recursive_check(left.left, right.right) and recursive_check(
                    left.right, right.left
                )

        return recursive_check(root.left, root.right)


    diameter = 0
    def diameterOfBinaryTree(self, root: Optional[TreeNode]) -> int:
        def postorder_traversal(node):
            if not node:
                return 0
            l_depth = postorder_traversal(node.left)
            r_depth = postorder_traversal(node.right)
            self.diameter = max(self.diameter, l_depth + r_depth + 1)
            return max(l_depth, r_depth) + 1

        postorder_traversal(root)

        return self.diameter - 1

    def levelOrder(self, root):
        if not root:
            return []

        result = []
        queue = deque([root])
        while queue:
            level_size = len(queue)
            level_res = []
            for _ in range(level_size):
                node = queue.popleft()
                level_res.append(node.val)
                if node.left:
                    queue.append(node.left)
                if node.right:
                    queue.append(node.right)
            result.append(level_res)
        return result

    def sortedArrayToBST(self, nums: List[int]) -> Optional[TreeNode]:
        """
        平衡二叉树：左右子树高度差不大于1
        二叉搜索树：左子树所有节点值小于node，右子树所有节点值大于node
        """
        if not nums:
            return None

        mid_idx = len(nums) // 2
        mid = nums[mid_idx]
        root = TreeNode(val=mid)
        root.left = self.sortedArrayToBST(nums[:mid_idx])
        root.right = self.sortedArrayToBST(nums[mid_idx + 1 :])
        return root

    def isValidBST(self, root: Optional[TreeNode]) -> bool:
        def helper(node, low, high):
            if not node:
                return True
            if not (low < node.val < high):
                return False
            return helper(node.left, low, node.val) and helper(
                node.right, node.val, high
            )

        return helper(root, -math.inf, math.inf)

    def kthSmallest(self, root: Optional[TreeNode], k: int) -> int:
        """中序遍历即为升序列表"""
        self.count = 0
        self.res = None

        def inorder_traversal(node):
            if not node or self.res is not None:
                return
            inorder_traversal(node.left)
            self.count += 1
            if self.count == k:
                self.res = node.val
                return
            inorder_traversal(node.right)

        inorder_traversal(root)
        return self.res

    def rightSideView(self, root: Optional[TreeNode]) -> List[int]:
        if not root:
            return []

        result = []
        queue = deque([root])
        while queue:
            level_size = len(queue)
            for idx in range(level_size):
                node = queue.popleft()
                if idx == level_size-1:
                    result.append(node.val)
                if node.left:
                    queue.append(node.left)
                if node.right:
                    queue.append(node.right)
        return result
    
    def buildTree(self, preorder: List[int], inorder: List[int]) -> Optional[TreeNode]:
        if preorder[0] == -1 or inorder[0] == -1:
            return TreeNode(-1)
        def build_tree(pre_l, in_l):
            if not pre_l or not in_l:
                return
            root_val = pre_l[0]
            root = TreeNode(root_val)
            root_idx = in_l.index(root_val)
            # 分隔左右子树的先序和中序序列
            left_in = in_l[:root_idx]
            right_in = in_l[root_idx+1:]
            left_pre = pre_l[1:1+len(left_in)]
            right_pre = pre_l[1+len(left_in):]
            # 构建左右子树
            root.left = build_tree(left_pre, left_in)
            root.right = build_tree(right_pre, right_in)
            return root
        
        return build_tree(preorder, inorder)

    def pathSum(self, root: Optional[TreeNode], targetSum: int) -> int:
        def dfs(node, curr_sum, prefix_sums):
            if not node:
                return 0
            curr_sum += node.val
            # 统计以当前节点为终点、和为targetSum的路径数
            count = prefix_sums.get(curr_sum - targetSum, 0)
            # 当前前缀和加入哈希表
            prefix_sums[curr_sum] = prefix_sums.get(curr_sum, 0) + 1
            # 递归左右子树
            count += dfs(node.left, curr_sum, prefix_sums)
            count += dfs(node.right, curr_sum, prefix_sums)
            # 回溯，撤销当前节点的前缀和
            prefix_sums[curr_sum] -= 1
            return count
        # 初始前缀和为0出现1次
        return dfs(root, 0, {0: 1})
    
    max_path_sum = -inf
    def maxPathSum(self, root: Optional[TreeNode]) -> int:
        def dfs(node):
            if not node:
                return 0
            
            left_max_path_sum = dfs(node.left)
            right_max_path_sum = dfs(node.right)

            left_gain = max(0, left_max_path_sum)
            right_gain = max(0, right_max_path_sum)

            curr_max_path = node.val + left_gain + right_gain
            self.max_path_sum = max(self.max_path_sum, curr_max_path)
            # 向上汇报时路径不能分叉，只能选择一条
            return node.val + max(left_gain, right_gain)
        
        dfs(root)
        return self.max_path_sum
    
    

## 图论

In [ ]:
class Solution:
    def numIslands(self, grid: List[List[str]]) -> int:
        islands = 0

        if not grid or not grid[0]:
            return islands

        def dfs_recursive(grid, row, col):
            if (
                row < 0
                or row >= len(grid)
                or col < 0
                or col >= len(grid[0])
                or grid[row][col] != "1"
            ):
                return

            grid[row][col] = "0"

            dfs_recursive(grid, row - 1, col)
            dfs_recursive(grid, row + 1, col)
            dfs_recursive(grid, row, col - 1)
            dfs_recursive(grid, row, col + 1)

        rows, cols = len(grid), len(grid[0])
        for i in range(rows):
            for j in range(cols):
                if grid[i][j] == "1":
                    dfs_recursive(grid, i, j)
                    islands += 1

        return islands

    def orangesRotting(self, grid: List[List[int]]) -> int:
        fresh = 0
        min_time = 0
        queue = deque()

        rows, cols = len(grid), len(grid[0])
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        if not grid or not grid[0]:
            return min_time

        # 初始化，入队烂橘子，统计好橘子
        for row in range(rows):
            for col in range(cols):
                if grid[row][col] == 2:

                    queue.append((row, col))
                if grid[row][col] == 1:
                    fresh += 1

        if not fresh:
            return 0

        while queue:
            level_size = len(queue)

            # 在每一分钟的开始，如果队列非空，就意味着这一分钟会有腐烂发生
            # 所以在这里增加时间是正确的。
            # 如果是最后一轮腐烂，即使没有新的橘子被加入队列，也代表了这一分钟的消耗。
            if fresh > 0:
                min_time += 1

            for _ in range(level_size):
                r, c = queue.popleft()
                for dr, dc in directions:
                    row, col = r + dr, c + dc

                    if 0 <= row < rows and 0 <= col < cols and grid[row][col] == 1:
                        grid[row][col] = 2
                        queue.append((row, col))
                        fresh -= 1

        # bfs结束，检查好橘子
        if fresh == 0:
            return min_time
        else:
            return -1

    def canFinish(self, numCourses: int, prerequisites: List[List[int]]) -> bool:
        # 构建邻接表
        graph = [[] for _ in range(numCourses)]
        for course, pre in prerequisites:
            graph[course].append(pre)

        # 0: 未访问, 1: 正在访问(在递归栈中), 2: 已完成访问
        visited = [0] * numCourses
        def dfs(course):
            # 正在访问中，说明有环
            if visited[course] == 1:
                return False

            # 完成访问
            if visited[course] == 2:
                return True

            # 标记为正在访问
            visited[course] = 1

            # 递归访问所有先修课程
            for pre in graph[course]:
                if not dfs(pre):
                    return False

            visited[course] = 2
            return True

        # 检查每门课程
        for i in range(numCourses):
            if not dfs(i):
                return False

        return True


# 前缀树
class Node:
    def __init__(self):
        self.son = {}
        self.end = False


class Trie:
    def __init__(self):
        self.root = Node()

    def insert(self, word: str) -> None:
        curr = self.root
        for c in word:
            if c not in curr.son:
                curr.son[c] = Node()
            curr = curr.son[c]
        curr.end = True

    def find(self, word: str):
        curr = self.root
        for c in word:
            if c not in curr.son:
                return 0
            curr = curr.son[c]
        return 2 if curr.end else 1

    def search(self, word: str) -> bool:
        return self.find(word) == 2

    def startsWith(self, prefix: str) -> bool:
        return self.find(prefix) != 0

## 双指针

In [ ]:
class Solution:
    def trap(self, height):
        if not height or len(height) < 3:
            return 0
        
        # 初始化左右指针
        left = 0                 # 左指针，从数组开始
        right = len(height) - 1  # 右指针，从数组末尾
        left_max = 0             # 记录左侧最大高度
        right_max = 0            # 记录右侧最大高度
        water = 0                # 累计接雨水量
        
        # 当左右指针未相遇时继续
        while left < right:
            # 比较左右指针指向的高度
            if height[left] < height[right]:
                # 左侧高度较小，处理左侧
                if height[left] >= left_max:
                    # 更新左侧最大高度
                    left_max = height[left]
                else:
                    # 当前位置可以接雨水
                    # 雨水量 = 左侧最大高度 - 当前高度
                    water += left_max - height[left]
                left += 1  # 左指针右移
            else:
                # 右侧高度较小或相等，处理右侧
                if height[right] >= right_max:
                    # 更新右侧最大高度
                    right_max = height[right]
                else:
                    # 当前位置可以接雨水
                    # 雨水量 = 右侧最大高度 - 当前高度
                    water += right_max - height[right]
                right -= 1  # 右指针左移
        
        return water

## 子串

In [ ]:
class Solution:
    def maxSlidingWindow(self, nums: List[int], k: int) -> List[int]:
        # 使用双端队列存储数组索引，保持队列中索引对应的值单调递减
        # 保证队列中队首索引对应的值始终是最大值
        dq = deque()
        result = []

        # 窗口边界：[i-k+1 : i]
        for i in range(len(nums)):
            # 移除队列中超出窗口范围的索引
            while dq and dq[0] < (i - k + 1):
                dq.popleft()

            # 维护队列单调性，移除所有小于当前元素的索引
            while dq and nums[dq[-1]] < nums[i]:
                dq.pop()

            # 将当前索引加入队列
            dq.append(i)

            # 当窗口大小达到k时，开始记录最大值
            if i >= k - 1:
                # 队列头部索引对应的值就是当前窗口的最大值
                result.append(nums[dq[0]])

        return result

    def minWindow(self, s: str, t: str) -> str:
        if not s or not t or len(s) < len(t):
            return ""

        target_counts = collections.Counter(t)
        window = defaultdict(int)
        left = right = 0
        matched_count = 0

        min_len = inf
        start_idx = 0

        while right < len(s):
            char_r = s[right]
            right += 1

            # 更新窗口数据
            if char_r in target_counts:
                window[char_r] += 1
                # 如果当前字符的计数首次达到需求数量
                if window[char_r] == target_counts[char_r]:
                    matched_count += 1

            # left指针收缩：窗口已经覆盖所有t中字符
            while matched_count == len(target_counts):
                # 记录当前窗口长度，与min len比较并更新
                curr_len = right - left
                if curr_len < min_len:
                    min_len = curr_len
                    start_idx = left
                
                char_l = s[left]
                left += 1

                if char_l in target_counts:
                    # 注意：这里是先判断是否减少了 match_count，再减少 window[char_l]
                    # 因为如果 window[char_l] 减少后小于 needs[char_l]，则 match_count 也要相应减少
                    if window[char_l] == target_counts[char_l]: # 如果当前字符的计数即将低于需求数量
                        matched_count -= 1
                    window[char_l] -= 1

        if min_len == -1:
            return ""
        else:
            return s[start_idx: start_idx+min_len]

## 普通数组

In [ ]:
class Solution:
    def firstMissingPositive(self, nums: List[int]) -> int:
        n = len(nums)

        # 非正数和大于n的数替换
        for i in range(n):
            if nums[i] <= 0 or nums[i] > n:
                nums[i] = inf

        # 使用数组索引作为哈希表，标记数字的存在性
        # 都标记为 -num , 作为存在的证明
        for i in range(n):
            num = abs(nums[i])
            if num <= n:
                nums[num - 1] = -abs(nums[num - 1])

        # 找到第一个正数位置，即为缺失的第一个正数
        for i in range(n):
            if nums[i] > 0:
                return i + 1

        return n + 1

## 回溯

In [ ]:
class Solution:
    def permute(self, nums: List[int]) -> List[List[int]]:
        result = []
        path = []
        used = [False] * len(nums)

        def backtrack():
            # 递归终止：当前路径长度 == 数组长度
            if len(path) == len(nums):
                # 必须添加副本，因为path后续会被修改
                result.append(path[:])
                return

            for i in range(len(nums)):
                if used[i]:
                    continue

                path.append(nums[i])
                used[i] = True

                # 进入下一层决策树
                backtrack()

                # 撤销选择：回溯到上一状态
                path.pop()
                used[i] = False

        backtrack()
        return result

    def subsets(self, nums: List[int]) -> List[List[int]]:
        result = []
        sub = []

        def backtrack(start_index):
            result.append(sub[:])

            for i in range(start_index, len(nums)):
                sub.append(nums[i])
                backtrack(i + 1)
                sub.pop()

        backtrack(0)
        return result

    def letterCombinations(self, digits: str) -> List[str]:
        if not digits:
            return []
        DIG_MAP = {
            "2": ["a", "b", "c"],
            "3": ["d", "e", "f"],
            "4": ["g", "h", "i"],
            "5": ["j", "k", "l"],
            "6": ["m", "n", "o"],
            "7": ["p", "q", "r", "s"],
            "8": ["t", "u", "v"],
            "9": ["w", "x", "y", "z"],
        }
        result = []
        path = []

        def backtrack(index):
            if len(path) == len(digits):
                result.append("".join(path))
                return

            for char in DIG_MAP[digits[index]]:
                path.append(char)
                backtrack(index + 1)
                path.pop()

        backtrack(0)
        return result

    def combinationSum(self, candidates: List[int], target: int) -> List[List[int]]:
        result = []
        path = []

        def backtrack(start, curr_sum):
            if curr_sum == target:
                result.append(path[:])
                return
            elif curr_sum > target:
                return

            for i in range(start, len(candidates)):
                path.append(candidates[i])
                backtrack(i, curr_sum + candidates[i])
                path.pop()

        backtrack(0, 0)
        return result

    def generateParenthesis(self, n: int) -> List[str]:
        result = []

        def backtrack(curr, left_count, right_count):
            if len(curr) == 2 * n:
                result.append(curr)
                return

            if left_count < n:
                backtrack(curr + "(", left_count + 1, right_count)

            if right_count < left_count:
                backtrack(curr + ")", left_count, right_count + 1)

        backtrack("", 0, 0)
        return result

    def exist(self, board: List[List[str]], word: str) -> bool:
        m, n = len(board), len(board[0])

        def dfs_recursive(i, j, k):
            # 边界检查和字符匹配检查
            if i < 0 or i >= m or j < 0 or j >= n or board[i][j] != word[k]:
                return False

            # 如果已经完整匹配
            if k == len(word) - 1:
                return True

            # 标记当前位置为已访问（临时修改）
            temp = board[i][j]
            board[i][j] = "#"

            # 检查四个方向
            found = (
                dfs_recursive(i + 1, j, k + 1)
                or dfs_recursive(i - 1, j, k + 1)
                or dfs_recursive(i, j + 1, k + 1)
                or dfs_recursive(i, j - 1, k + 1)
            )

            # 回溯：恢复当前位置的字符
            board[i][j] = temp

            return found

        k = 0
        for i in range(m):
            for j in range(n):
                if dfs_recursive(i, j, k):
                    return True

        return False
    
    def partition(self, s: str) -> List[List[str]]:
        result = []
        sub = []
        
        def is_palindrome(string):
            if not string:
                return False
            return string == string[::-1]
        
        def backtrack(index):
            if index == len(s):
                result.append(sub[:])
                return
            
            for i in range(index, len(s)):
                substring = s[index: i+1]
                if is_palindrome(substring):
                    sub.append(substring)
                    backtrack(i + 1)
                    sub.pop()

        backtrack(0)
        return result

    def solveNQueens(self, n: int) -> List[List[str]]:
        # 存储所有可行的解决方案
        result = []
        # 记录每行皇后所在的列位置，col[r]表示第r行的皇后在第col[r]列
        col = [0] * n

        # 深度优先搜索函数
        # r: 当前正在处理的行
        # rest_cols: 剩余可选的列集合，用集合数据结构便于快速删除和查找
        def dfs(r, rest_cols):
            # 递归终止条件：当处理完所有行（r == n）时，找到一个有效解
            if r == n:
                # 将解转换为题目要求的字符串格式
                # 对于每一行，生成一个长度为n的字符串：
                # '.'*c 表示皇后前面的空位
                # 'Q' 表示皇后位置
                # '.'*(n-c-1) 表示皇后后面的空位
                result.append(
                    ['.'*c + 'Q' + '.'*(n-c-1) for c in col]
                )
                return

            # 遍历当前行所有可选的列位置
            for c in rest_cols:
                # 检查当前位置是否合法（不与已放置的皇后冲突）
                # 对于每个已放置的皇后(R, col[R])，检查是否与当前皇后(r, c)冲突：
                # r+c != R+col[R] 检查是否在同一个主对角线（行索引与列索引之和相等）
                # r-c != R-col[R] 检查是否在同一个副对角线（行索引与列索引之差相等）
                # all(...) 确保与之前所有行的皇后都不冲突
                if all(r+c != R+col[R] and r-c!=R-col[R] for R in range(r)):
                    # 位置合法，将第r行的皇后放在第c列
                    col[r] = c
                    # 递归处理下一行，同时从可选列集合中移除已使用的列c
                    dfs(r+1, rest_cols-{c})

        # 从第0行开始搜索，初始可选列集合为所有列{0,1,2,...,n-1}
        dfs(0, set(range(n)))
        return result

## 二分查找

In [ ]:
class Solution:
    def searchRange(self, nums: List[int], target: int) -> List[int]:
        start = bisect.bisect_left(nums, target)
        if start == len(nums) or nums[start] != target:
            return [-1, -1]
        end = bisect.bisect_right(nums, target) - 1
        return [start, end]

    def searchRange(self, nums: List[int], target: int) -> List[int]:
        # 寻找左边界
        def find_first_position(nums, target):
            left, right = 0, len(nums) - 1
            first_pos = -1
            while left <= right:
                mid = left + (right - left) // 2
                if nums[mid] > target:
                    right = mid - 1
                elif nums[mid] < target:
                    left = mid + 1
                else:  # nums[mid] == target
                    first_pos = mid  # 记录当前位置
                    right = mid - 1  # 尝试在左半部分继续寻找更早的位置
            return first_pos

        # 寻找右边界
        def find_last_position(nums, target):
            left, right = 0, len(nums) - 1
            last_pos = -1
            while left <= right:
                mid = left + (right - left) // 2
                if nums[mid] > target:
                    right = mid - 1
                elif nums[mid] < target:
                    left = mid + 1
                else:  # nums[mid] == target
                    last_pos = mid  # 记录当前位置
                    left = mid + 1  # 尝试在右半部分继续寻找更晚的位置
            return last_pos

        first = find_first_position(nums, target)

        # 如果连左边界都找不到，说明元素不存在，直接返回
        if first == -1:
            return [-1, -1]

        last = find_last_position(nums, target)

        return [first, last]

    def search(self, nums: List[int], target: int) -> int:
        left, right = 0, len(nums) - 1
        while left <= right:
            mid = (left + right) // 2
            if nums[mid] == target:
                return mid

            # 左半部分有序
            if nums[left] <= nums[mid]:
                if nums[left] <= target < nums[mid]:
                    right = mid - 1
                else:
                    left = mid + 1
            # 右半部分有序
            else:
                if nums[mid] < target <= nums[right]:
                    left = mid + 1
                else:
                    right = mid - 1
        return -1

    def findMin(self, nums: List[int]) -> int:
        left, right = 0, len(nums) - 1
        if nums[left] < nums[right]:
            return nums[left]

        while left < right:
            mid = left + (right - left) // 2

            if nums[mid] < nums[right]:
                # mid在右侧的有序部分，minimum在mid或者左侧
                right = mid
            elif nums[mid] > nums[right]:
                # mid在左侧的有序部分，minimum在右侧
                left = mid + 1
            else:  # nums[mid]==nums[right]
                # 无法判断在哪一侧，但是可以排除right
                right -= 1

        return nums[left]


    def findMedianSortedArrays(self, nums1: List[int], nums2: List[int]) -> float:
        # 获取两个数组的长度
        m = len(nums1)
        n = len(nums2)
        # 处理边界情况
        if m == 0:
            if n % 2 == 0:
                return (nums2[n // 2 - 1] + nums2[n // 2]) / 2
            else:
                # 奇数个元素：返回中间元素
                return nums2[n // 2]
        if n == 0:
            if m % 2 == 0:
                return (nums1[m // 2 - 1] + nums1[m // 2]) / 2
            else:
                # 奇数个元素：返回中间元素
                return nums1[m // 2]

        # 合并两个已排序数组
        merged = []
        i = j = 0  # 分别指向两个数组的当前索引

        # 按顺序合并两个数组，直到其中一个数组遍历完
        while i < m and j < n:
            if nums1[i] < nums2[j]:
                merged.append(nums1[i])
                i += 1
            else:
                merged.append(nums2[j])
                j += 1

        # 将nums1中剩余的元素添加到合并数组中
        while i < m:
            merged.append(nums1[i])
            i += 1

        # 将nums2中剩余的元素添加到合并数组中
        while j < n:
            merged.append(nums2[j])
            j += 1

        # 计算合并后数组的总长度
        total_len = m + n

        # 根据总长度的奇偶性计算中位数
        if total_len % 2 == 0:
            # 偶数个元素：返回中间两个元素的平均值
            return (merged[total_len // 2 - 1] + merged[total_len // 2]) / 2
        else:
            # 奇数个元素：返回中间元素
            return merged[total_len // 2]

## 栈

In [ ]:
class Solution:
    def isValid(self, s: str) -> bool:
        mapping = {'(':')', '{':'}', '[':']', -1:-1}
        stack = [-1]
        for char in s:
            if char in mapping:
                stack.append(char)
            elif char != mapping[stack.pop()]:
                return False
            
        return len(stack) == 1